In [3]:
from ultralytics import YOLO
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import urllib
import itertools
import random, os, glob
from imutils import paths
from sklearn.utils import shuffle
from urllib.request import urlopen

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import  ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, SpatialDropout2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

2023-02-25 13:17:17.432769: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
target_size = (224, 224)
waste_labels = {"cardboard":0, "glass":1, "metal":2, "paper":3, "plastic":4, "trash":5}
dir_path = "./GarbageClassification"

In [7]:
def load_dataset(path):
  x = []
  labels = []
  image_paths = sorted(list(paths.list_images(path)))
  for image_path in image_paths:
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    x.append(img)
    label = image_path.split(os.path.sep)[-2]
    labels.append(waste_labels[label])
  x, labels = shuffle(x, labels, random_state=42)
  input_shape = (np.array(x[0]).shape[1], np.array(x[0]).shape[1], 3)
  print("X shape: ", np.array(x).shape)
  print(f"Number of Labels: {len(np.unique(labels))} , Number of Observation: {len(labels)}")
  print("Input Shape: ", input_shape)
  return x, labels, input_shape

In [8]:
x, labels, input_shape = load_dataset(dir_path)

X shape:  (2527, 224, 224, 3)
Number of Labels: 6 , Number of Observation: 2527
Input Shape:  (224, 224, 3)


In [11]:
def visualization_img(image_batch, labels, num_of_img):
  plt.figure(figsize=(10,10))
  for n in range(num_of_img):
    ax = plt.subplot(5,5,n+1)
    plt.imshow(image_batch[n])
    plt.title(np.array(list(waste_labels.keys()))[to_categorical(labels, num_classes=len(np.unique(labels)))[n] == 1][0].title())
    plt.axis("off")

In [12]:
visualization_img(x, labels, 10)

Found 2275 images belonging to 1 classes.
Found 252 images belonging to 1 classes.


In [21]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (224, 224, 3)) ,
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
    tf.keras.layers.Dropout(0.1,seed = 2019),
    tf.keras.layers.Dense(400,activation ="relu"),
    tf.keras.layers.Dropout(0.3,seed = 2019),
    tf.keras.layers.Dense(300,activation="relu"),
    tf.keras.layers.Dropout(0.4,seed = 2019),
    tf.keras.layers.Dense(200,activation ="relu"),
    tf.keras.layers.Dropout(0.2,seed = 2019),
    tf.keras.layers.Dense(6,activation = "softmax")   #Adding the Output Layer
])

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 111, 111, 16)     0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 26, 26, 64)      

In [25]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
adam=Adam(lr=0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])

In [23]:
bs=30         #Setting batch size
train_dir = "./GarbageClassification/Data/Train"   #Setting training directory
validation_dir = "./GarbageClassification/Data/Valid"   #Setting testing directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
# Flow training images in batches of 20 using train_datagen generator
#Flow_from_directory function lets the classifier directly identify the labels from the name of the directories the image lies in
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(224,224))
# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(224,224))

Found 2279 images belonging to 6 classes.
Found 250 images belonging to 6 classes.


In [27]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    steps_per_epoch=150 // bs,
                    epochs=100,
                    validation_steps=50 // bs,
                    verbose=2)

Epoch 1/100
5/5 - 6s - loss: 1.3225 - acc: 0.4600 - val_loss: 1.2927 - val_acc: 0.4667 - 6s/epoch - 1s/step
Epoch 2/100
5/5 - 6s - loss: 1.3478 - acc: 0.4867 - val_loss: 1.2567 - val_acc: 0.4333 - 6s/epoch - 1s/step
Epoch 3/100
5/5 - 6s - loss: 1.3936 - acc: 0.4161 - val_loss: 1.3429 - val_acc: 0.5333 - 6s/epoch - 1s/step
Epoch 4/100
5/5 - 6s - loss: 1.2795 - acc: 0.4933 - val_loss: 1.2528 - val_acc: 0.4667 - 6s/epoch - 1s/step
Epoch 5/100
5/5 - 6s - loss: 1.2897 - acc: 0.4667 - val_loss: 1.5128 - val_acc: 0.3333 - 6s/epoch - 1s/step
Epoch 6/100
5/5 - 6s - loss: 1.3286 - acc: 0.4933 - val_loss: 1.1825 - val_acc: 0.4333 - 6s/epoch - 1s/step
Epoch 7/100
5/5 - 9s - loss: 1.3321 - acc: 0.4867 - val_loss: 1.4587 - val_acc: 0.4000 - 9s/epoch - 2s/step
Epoch 8/100
5/5 - 8s - loss: 1.2052 - acc: 0.5200 - val_loss: 1.2881 - val_acc: 0.3000 - 8s/epoch - 2s/step
Epoch 9/100
5/5 - 9s - loss: 1.4153 - acc: 0.4333 - val_loss: 1.5266 - val_acc: 0.3000 - 9s/epoch - 2s/step
Epoch 10/100
5/5 - 7s - loss

Epoch 77/100
5/5 - 6s - loss: 0.5899 - acc: 0.7800 - val_loss: 1.1377 - val_acc: 0.6000 - 6s/epoch - 1s/step
Epoch 78/100
5/5 - 7s - loss: 0.6497 - acc: 0.7987 - val_loss: 0.8593 - val_acc: 0.7000 - 7s/epoch - 1s/step
Epoch 79/100
5/5 - 7s - loss: 0.6877 - acc: 0.8200 - val_loss: 1.2976 - val_acc: 0.6333 - 7s/epoch - 1s/step
Epoch 80/100
5/5 - 9s - loss: 0.8196 - acc: 0.7267 - val_loss: 1.0563 - val_acc: 0.7333 - 9s/epoch - 2s/step
Epoch 81/100
5/5 - 7s - loss: 0.7474 - acc: 0.7400 - val_loss: 1.0967 - val_acc: 0.5333 - 7s/epoch - 1s/step
Epoch 82/100
5/5 - 9s - loss: 0.7485 - acc: 0.7800 - val_loss: 0.8072 - val_acc: 0.7667 - 9s/epoch - 2s/step
Epoch 83/100
5/5 - 7s - loss: 0.6672 - acc: 0.7533 - val_loss: 0.8015 - val_acc: 0.6667 - 7s/epoch - 1s/step
Epoch 84/100
5/5 - 6s - loss: 0.6506 - acc: 0.7533 - val_loss: 0.8696 - val_acc: 0.7333 - 6s/epoch - 1s/step
Epoch 85/100
5/5 - 7s - loss: 0.5071 - acc: 0.8267 - val_loss: 1.1735 - val_acc: 0.6000 - 7s/epoch - 1s/step
Epoch 86/100
5/5 - 

In [28]:
def CNN_model_evaluate(model):
  loss, precision, recall, acc = model.evaluate(test_generator, batch_size=32)
  print("Test Accuracy: %.2f" % (100 * acc))
  print("Test Loss: %.2f" % (100 * loss))
  print("Test Precision: %.2f" % (100 * precision))
  print("Test Recall: %.2f" % (100 * recall))

  X_test, y_test = test_generator.next()
  y_pred = model.predict(X_test)
  y_pred = np.argmax(y_pred, axis=1)
  y_test = np.argmax(y_test, axis=1)
  target_names = list(waste_labels.keys())
  print(classification_report(y_test, y_pred, target_names=target_names))

  plt.figure(figsize=(20,5))
  plt.subplot(1,2,1)
  plt.plot(history.history["acc"], color="r", label="Training Accuracy")
  plt.plot(history.history["val_acc"], color="b", label="Validation Accuracy")
  plt.legend(loc="lower right")
  plt.xlabel("Epoch", fontsize=16)
  plt.ylabel("Accuracy", fontsize=16)
  plt.ylim([min(plt.ylim()),1])
  plt.title("Training and Validation Accuracy", fontsize=16)

  plt.subplot(1,2,2)
  plt.plot(history.history["loss"], color="r", label="Training Loss")
  plt.plot(history.history["val_loss"], color="b", label="Validation Loss")
  plt.legend(loc="upper right")
  plt.xlabel("Epoch", fontsize=16)
  plt.ylabel("Loss", fontsize=16)
  plt.ylim([0, max(plt.ylim())])
  plt.title("Training and Validation Loss", fontsize=16)

  return y_test, y_pred

In [29]:
model.save('./currentmodel.h5')

In [30]:
y_test, y_pred = CNN_model_evaluate(model)

FileNotFoundError: [Errno 2] No such file or directory: './GarbageClassification/Garbage classification/Garbage classification/cardboard/cardboard184.jpg'

In [ ]:
cm = confusion_matrix(y_test, y_pred)
def plot_confusion_matrix(cm, classes, normalize=False, title="Confusion Matrix", cmap=plt.cm.OrRd):
  if normalize:
    cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
  
  plt.figure(figsize=(8,6))
  plt.imshow(cm, interpolation="nearest", cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)
  fmt = ".2f" if normalize else "d"
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
  plt.tight_layout()
  plt.ylabel("True Labels", fontweight="bold")
  plt.xlabel("Predicted Labels", fontweight="bold")

plot_confusion_matrix(cm, waste_labels.keys())

In [31]:
def CNN_model_testing(path):
  img = image.load_img(path, target_size=(target_size))
  img = image.img_to_array(img, dtype=np.uint8)
  img = np.array(img)/255.0
  p = model.predict(img.reshape(1,224,224,3))
  predicted_class = np.argmax(p[0])
  return img, p, predicted_class

In [39]:
img, p, predicted_class = CNN_model_testing("./glass51.jpg")

1/1 [==============================] - 0s 44ms/step


array([[ 1.1965e-07,     0.99997,  1.1308e-05,   8.157e-08,  2.1339e-05,  4.2926e-08]], dtype=float32)